#### HDS5230-Week05- Dask Programming Assignment
##### Author: Wenshan Liu
##### Date: 2025-02-23

In [1]:
import pandas as pd
import dask.dataframe as dd


In [3]:
# Q1. Q2
# Extract the .csv file from the zipped archive, and load it into a Dask dataframe.

# Define the data types or it will cause the error
dtypes = {
    'aggregate': 'object',
    'cases': 'float64',
    'city': 'object',
    'population': 'float64'
}

#read the data
ddf = dd.read_csv('timeseries.csv', dtype=dtypes)

# check the first few rows
#print(ddf.head())

# check the data types
print(ddf.dtypes)

name                    string[pyarrow]
level                   string[pyarrow]
city                    string[pyarrow]
county                  string[pyarrow]
state                   string[pyarrow]
country                 string[pyarrow]
population                      float64
lat                             float64
long                            float64
url                     string[pyarrow]
aggregate               string[pyarrow]
tz                      string[pyarrow]
cases                           float64
deaths                          float64
recovered                       float64
active                          float64
tested                          float64
hospitalized                    float64
hospitalized_current            float64
discharged                      float64
icu                             float64
icu_current                     float64
growthFactor                    float64
date                    string[pyarrow]
dtype: object


In [4]:
# Q3.1 Create a new dataframe object that consists of samples (i.e., rows) corresponding to states in the US.

# filter the data from United States
us_states_df = ddf[
    (ddf['country'] == 'United States') & 
    (ddf['state'].notnull()) &
    (ddf['level'] == 'state')
]


print("data sample:")
print(us_states_df.head())

# get the unique states
unique_states = us_states_df['state'].unique().compute()
print("unique states:")
print(unique_states)

# get the number of states
print("number of states:")
print(len(unique_states))

data sample:


/Users/sandyliu/anaconda3/envs/Class_HDS5330/lib/python3.10/site-packages/dask/dataframe/io/csv.py:77: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
/Users/sandyliu/anaconda3/envs/Class_HDS5330/lib/python3.10/site-packages/dask/dataframe/io/csv.py:77: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


                          name  level  city county     state        country  \
37985  Virginia, United States  state  <NA>   <NA>  Virginia  United States   
37986  Virginia, United States  state  <NA>   <NA>  Virginia  United States   
37987  Virginia, United States  state  <NA>   <NA>  Virginia  United States   
37988  Virginia, United States  state  <NA>   <NA>  Virginia  United States   
37989  Virginia, United States  state  <NA>   <NA>  Virginia  United States   

       population      lat     long  \
37985   8535519.0  38.0035 -79.4585   
37986   8535519.0  38.0035 -79.4585   
37987   8535519.0  38.0035 -79.4585   
37988   8535519.0  38.0035 -79.4585   
37989   8535519.0  38.0035 -79.4585   

                                                     url  ... recovered  \
37985  https://covidtracking.com/api/v1/states/curren...  ...       NaN   
37986  https://covidtracking.com/api/v1/states/curren...  ...       NaN   
37987  https://covidtracking.com/api/v1/states/curren...  ...    

/Users/sandyliu/anaconda3/envs/Class_HDS5330/lib/python3.10/site-packages/dask/dataframe/io/csv.py:77: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


unique states:
0                         Virginia
1                       Washington
2                          Alabama
3                           Alaska
4                          Arizona
5                         Arkansas
6                       California
7                         Colorado
8                      Connecticut
9                         Delaware
10                Washington, D.C.
11                         Florida
12                         Georgia
13                          Hawaii
14                           Idaho
15                        Illinois
16                         Indiana
17                            Iowa
18                          Kansas
19                        Kentucky
20                       Louisiana
21                           Maine
22                        Maryland
23                   Massachusetts
24                        Michigan
25                       Minnesota
26                     Mississippi
27                        Missouri
28   

In [ ]:
# filter state manually (Since the data is still with the non-US states, so need to filter out the states that are not in the US)
us_states_df = us_states_df[us_states_df['state'] != 'Washington, D.C.']
us_states_df = us_states_df[us_states_df['state'] != 'Puerto Rico']
us_states_df = us_states_df[us_states_df['state'] != 'Guam']
us_states_df = us_states_df[us_states_df['state'] != 'Northern Mariana Islands']
us_states_df = us_states_df[us_states_df['state'] != 'Virgin Islands']
us_states_df = us_states_df[us_states_df['state'] != 'American Samoa']
us_states_df = us_states_df[us_states_df['state'] != 'Micronesia']
us_states_df = us_states_df[us_states_df['state'] != 'United States Virgin Islands']

unique_states = us_states_df['state'].unique().compute()

print(f"number of states: {len(unique_states)}")
print(unique_states)

/Users/sandyliu/anaconda3/envs/Class_HDS5330/lib/python3.10/site-packages/dask/dataframe/io/csv.py:77: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


number of states: 50
0           Virginia
1         Washington
2            Alabama
3             Alaska
4            Arizona
5           Arkansas
6         California
7           Colorado
8        Connecticut
9           Delaware
10           Florida
11           Georgia
12            Hawaii
13             Idaho
14          Illinois
15           Indiana
16              Iowa
17            Kansas
18          Kentucky
19         Louisiana
20             Maine
21          Maryland
22     Massachusetts
23          Michigan
24         Minnesota
25       Mississippi
26          Missouri
27           Montana
28          Nebraska
29            Nevada
30     New Hampshire
31        New Jersey
32        New Mexico
33          New York
34    North Carolina
35      North Dakota
36              Ohio
37          Oklahoma
38            Oregon
39      Pennsylvania
40      Rhode Island
41    South Carolina
42      South Dakota
43         Tennessee
44             Texas
45              Utah
46           

In [11]:
#3.2 During the time period 2020-Jan-01 to 2021-Feb-28, rank states in terms of their per-capita mortality?

#to get the mortality stats from 20200101 to 20210228
us_states_df['date'] = dd.to_datetime(us_states_df['date'])

filtered_df = us_states_df[
    (us_states_df['date'] >= '2020-01-01') & 
    (us_states_df['date'] <= '2021-02-28')
]

mortality_stats = filtered_df.groupby('state').agg({
    'deaths': 'sum',
    'population': 'mean'
}).compute()

#print(mortality_stats.head())

# calculate the per capita mortality
mortality_stats['per_capita_mortality'] = mortality_stats['deaths'] / mortality_stats['population']

# sort the mortality stats
mortality_ranking = mortality_stats.sort_values('per_capita_mortality', ascending=False)

#print(mortality_ranking.head())


/Users/sandyliu/anaconda3/envs/Class_HDS5330/lib/python3.10/site-packages/dask/dataframe/io/csv.py:77: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


In [13]:

# to get the per capita mortality and sort the states

mortality_stats['per_capita_mortality'] = mortality_stats['deaths'] / mortality_stats['population']
mortality_ranking = mortality_stats.sort_values('per_capita_mortality', ascending=False)
mortality_ranking

,deaths,population,per_capita_mortality
state,,,
New Jersey,861317.0,8882190.0,0.096971
Connecticut,277777.0,3565287.0,0.077912
Massachusetts,482980.0,6892503.0,0.070073
New York,1358824.0,19453561.0,0.069850
Louisiana,213399.0,4648794.0,0.045904
Rhode Island,46435.0,1059361.0,0.043833
Michigan,419841.0,9986857.0,0.042039
Illinois,382919.0,12671821.0,0.030218
Pennsylvania,378963.0,12801989.0,0.029602


In [14]:

"""
# Q3.3
During the same time period, compute the case fatality rate (CFR) per month, 
using one of the approaches defined in this scientific brief from the World Health Organization: https://www.who.int/news-room/commentaries/detail/estimating-mortality-from-covid-19Links to an external site.. 
"""

# 3. to get the CFR per month
# add the year_month column
filtered_df['year_month'] = filtered_df['date'].dt.to_period('M')

# to get the CFR per month (using the WHO method)
monthly_cfr = filtered_df.groupby(['state', 'year_month']).agg({
    'deaths': 'sum',
    'cases': 'sum'
}).compute()

monthly_cfr['cfr'] = monthly_cfr['deaths'] / monthly_cfr['cases']

# reshape the data to a matrix and sort the columns
cfr_matrix = monthly_cfr['cfr'].unstack()
cfr_matrix =cfr_matrix.sort_index(axis=1)  # sort by month
cfr_matrix

/Users/sandyliu/anaconda3/envs/Class_HDS5330/lib/python3.10/site-packages/dask/dataframe/io/csv.py:77: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)


year_month,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07
state,,,,,,,
Virginia,NaN,NaN,0.014071,0.030015,0.033158,0.028627,0.028396
Washington,0.0,0.016667,0.049093,0.051323,0.053980,0.043972,0.039423
Alabama,NaN,NaN,0.005324,0.029829,0.038967,0.029617,0.023956
Alaska,NaN,NaN,0.000000,0.026795,0.024091,0.013322,0.014205
Arizona,0.0,0.000000,0.000000,0.029733,0.039077,0.004230,0.019471
Arkansas,NaN,NaN,0.009231,0.018736,0.021011,0.015134,0.012545
California,0.0,0.000000,0.020168,0.034800,0.039834,0.031787,0.025430
Colorado,NaN,NaN,0.018294,0.043694,0.053447,0.054194,0.050624
Connecticut,NaN,NaN,0.018184,0.064257,0.089880,0.093442,0.092793


In [15]:

# Q3.4 Using this matrix as input, compute the ranking of states, on how the CFR rate changed over time. This computation will involve an aggregation of of month-to-month changes in CFR. Note that some of these month-to-month changes can be positive (CFR increases from a previous month to the current month) or negative (CFR decrease from a previous month to the current month), or zero (no change in CFR). 
# You need to aggregate these individual changes, across all time periods.

# calculate the monthly change
cfr_changes = cfr_matrix.diff(axis=1)

# to get the CFR change rate
#total_change = cfr_changes.abs().sum(axis=1)  # Use the absolute value of the changes
total_change = cfr_changes.sum(axis=1)  # the overall change of CFR
cfr_change_ranking = total_change.sort_values(ascending=False)

# show the result

print("================================================")
print("CFR change rate ranking：")
print(cfr_change_ranking)



CFR change rate ranking：
state
New Jersey        0.083710
Connecticut       0.074609
Massachusetts     0.074312
Pennsylvania      0.064987
New Hampshire     0.061563
Michigan          0.061151
New York          0.058222
Rhode Island      0.055043
Illinois          0.048020
Maryland          0.044210
Minnesota         0.039993
Washington        0.039423
Ohio              0.037463
New Mexico        0.033150
Missouri          0.033118
Colorado          0.032330
West Virginia     0.030102
Delaware          0.029754
Indiana           0.029178
Maine             0.026348
California        0.025430
Wisconsin         0.023905
Mississippi       0.022253
Oregon            0.022106
North Dakota      0.020217
Montana           0.020157
Arizona           0.019471
Alabama           0.018633
North Carolina    0.016560
Iowa              0.014925
Kentucky          0.014807
Nebraska          0.014434
Virginia          0.014326
Alaska            0.014205
Texas             0.014024
Louisiana         0.0128

Q4: For each of the above operations, explain using appropriate reasoning, whether using a parallelized and/or distributed way of performing the computation makes sense. </br>

1. create data frame for each state --> Parallel processing </br>
Reason: Since data filter is based on the state, and the filter operation is independent of each other, so it is suitable for parallel processing. </br>


2. calculate the per capita mortality --> Parallel processing or Distributed </br>
Reason: Since the calculation of per capita mortality is also independent of each other so it is suitable for parallel processing. On the other hand, the aggregation operation also can be distributed by using dask distributed manner.
If the data is small, it is suitable to use parallel way to perform the computation. </br>

3. calculate the CFR --> Parallel processing or Distributed </br>
Reason: Since the calculation of CFR is also independent of each other so it is suitable for parallel processing. 
On the other hand, the aggregation operation also can be distributed by using dask distributed manner.
If the data is large, it is suitable to use distributed way to perform the computation.</br>
   
4. calculate the CFR change rate --> Parallel processing or Distributed </br>
Reason: Since the calculation of CFR change rate is also independent of each other so it is suitable for parallel processing. 
On the other hand, the aggregation operation also can be distributed by using dask distributed manner.
Since this matrix is small (only 50x14), it is suitable to use parallel way to perform the computation. </br>   